# Experiments querying events processed by AI



In [ ]:
DEVICE_NAME = "axis-p3827-front-far"

START_DATE = "2025-05-10T00:00:00.000000+00:00"
END_DATE = "2025-05-10T23:59:59.000000+00:00"

CLASSES_TO_QUERY = {"person", "dog", "bird"}

import boto3
from boto3.dynamodb.conditions import Key
from datetime import datetime
from pprint import pprint

# Initialize DynamoDB
dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")

table = dynamodb.Table("events")

all_items = []
last_key = None

while True:
    if last_key:
        response = table.query(
            KeyConditionExpression=Key("device_id").eq(DEVICE_NAME) &
                                     Key("event_timestamp").between(START_DATE, END_DATE),
            ExclusiveStartKey=last_key
        )
    else:
        response = table.query(
            KeyConditionExpression=Key("device_id").eq(DEVICE_NAME) &
                                     Key("event_timestamp").between(START_DATE, END_DATE)
        )

    all_items.extend(response.get("Items", []))

    last_key = response.get("LastEvaluatedKey")
    if not last_key:
        break

print(f"Total items: {len(all_items)}")
all_items

In [ ]:
# now, filter the items that have an intersection with the classes we are interested in
filtered_items = []
for item in all_items:
    if "seen_classes" in item and item["seen_classes"]:
        seen_classes = set(item["seen_classes"])
        if seen_classes.intersection(CLASSES_TO_QUERY):
            filtered_items.append(item)

print(f"Filtered items: {len(filtered_items)}")